In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
DATA_PATH = "/Users/mdong/dataScience/projects-ml/ca-waste/" + "data/"

# Forecasting waste production into the future


In [3]:
complete_feature_df = pd.read_csv(DATA_PATH + "complete_feature_df.csv")
complete_feature_df.set_index("Year", inplace=True)
complete_feature_df.head()

,Waste Produced (Tons),County,Population,Electricity Usage (GWh),Mobile Homes (units),Multi-Family (units),Single Family (units)
Year,,,,,,,
2000.0,1676429.25,Alameda,1443939.0,2926.106226,7631,203132,328399
2000.0,745.00,Alpine,1208.0,6.247035,62,561,883
2000.0,41059.90,Amador,35100.0,127.238094,1488,913,12563
2000.0,203896.87,Butte,203171.0,705.766172,14199,17317,53845
2000.0,34110.44,Calaveras,40554.0,173.578409,2235,856,19777


In [34]:
def forecast_preprocessing(county):
    """Aligns year n-1 features with year n.  e.g. 2019 waste produced should go with 2018 features
    """
    county_df = complete_feature_df.loc[complete_feature_df["County"] == 'Alameda']
    
    target = county_df[["Waste Produced (Tons)"]]
    feature_df = county_df.drop(columns="Waste Produced (Tons)")
    
    # shift features down a year
    shifted_feature_df = feature_df.shift(-1)
    shifted_feature_df.dropna(how="all", inplace=True)
    shifted_feature_df.drop(columns=["County"], inplace=True)
    
    # "shift" target by dropping earliest year
    min_year = target.index.min()
    target = target.drop(min_year)
    
    assert len(target) == len(shifted_feature_df), "Length of features and target need to be equal"
    
    return shifted_feature_df, target

In [88]:
alameda_features, alameda_target = forecast_preprocessing("Alameda")

In [36]:
alameda_features

,Population,Electricity Usage (GWh),Mobile Homes (units),Multi-Family (units),Single Family (units)
Year,,,,,
2000.0,1457185.0,2745.201964,7661.0,204237.0,332390.0
2001.0,1467063.0,2811.610305,7681.0,205444.0,335234.0
2002.0,1467892.0,2937.070283,7695.0,207168.0,338009.0
2003.0,1466407.0,2897.764371,7694.0,208691.0,340095.0
2004.0,1462736.0,3025.149538,7710.0,210656.0,342847.0
2005.0,1462371.0,3118.114844,7741.0,212381.0,345393.0
2006.0,1470622.0,2971.652195,7764.0,214253.0,347560.0
2007.0,1484085.0,2996.061668,7783.0,217324.0,349637.0
2008.0,1497799.0,3016.259087,7809.0,219394.0,351637.0


In [37]:
alameda_target

,Waste Produced (Tons)
Year,
2001.0,1629208.38
2002.0,1596803.00
2003.0,1585190.61
2004.0,1692478.59
2005.0,1659523.24
2006.0,1656037.42
2007.0,1551102.75
2008.0,1337915.56
2009.0,1246311.26


In [38]:
max_feature_year = alameda_features.index.max()
max_target_year = alameda_target.index.max()
print(max_feature_year, max_target_year)

2018.0 2019.0


In [47]:
X_train, y_train = alameda_features.loc[alameda_features.index!=max_feature_year], alameda_target[alameda_target.index!=max_target_year]
X_test, y_test = alameda_features.loc[alameda_features.index==max_feature_year], alameda_target[alameda_target.index==max_target_year]

In [48]:
assert 2019 not in X_train.index.unique(), f"{max_target_year} should not be in the training data"
assert 2019 == y_test.index.unique(), f"{max_target_year} should be the test data"

In [49]:
X_train.head()

,Population,Electricity Usage (GWh),Mobile Homes (units),Multi-Family (units),Single Family (units)
Year,,,,,
2000.0,1457185.0,2745.201964,7661.0,204237.0,332390.0
2001.0,1467063.0,2811.610305,7681.0,205444.0,335234.0
2002.0,1467892.0,2937.070283,7695.0,207168.0,338009.0
2003.0,1466407.0,2897.764371,7694.0,208691.0,340095.0
2004.0,1462736.0,3025.149538,7710.0,210656.0,342847.0


In [50]:
y_train.head()

,Waste Produced (Tons)
Year,
2001.0,1629208.38
2002.0,1596803.00
2003.0,1585190.61
2004.0,1692478.59
2005.0,1659523.24


In [51]:
random_forest_model = RandomForestRegressor()

possible_hyperparams = { 
    'n_estimators': [25, 50],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(5,10)]
}

grid_search = GridSearchCV(estimator=random_forest_model, param_grid=possible_hyperparams, cv=5, scoring='neg_mean_absolute_error')

In [52]:
grid_search.fit(X_train, y_train.values.ravel()) # need to call this ravel function because of https://stackoverflow.com/questions/34165731/a-column-vector-y-was-passed-when-a-1d-array-was-expected
best_score = -grid_search.best_score_ # needs to be negated because of https://stackoverflow.com/questions/21443865/scikit-learn-cross-validation-negative-values-with-mean-squared-error
best_rf_model = grid_search.best_estimator_
print(best_score, best_rf_model)

86138.91634444437 RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
                      max_features='log2', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)


/Users/mdong/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [53]:
X_test

,Population,Electricity Usage (GWh),Mobile Homes (units),Multi-Family (units),Single Family (units)
Year,,,,,
2018.0,1664783.0,3064.781376,7859.0,232321.0,365797.0


In [66]:
alameda_predicted = best_rf_model.predict(X_test)
alameda_predicted

array([1339123.2278])

In [94]:
alameda_target.values

array([[1629208.38  ],
       [1596803.    ],
       [1585190.61  ],
       [1692478.59  ],
       [1659523.24  ],
       [1656037.42  ],
       [1551102.75  ],
       [1337915.56  ],
       [1246311.26  ],
       [1152323.9   ],
       [1090342.95  ],
       [1147859.1   ],
       [1143318.37  ],
       [1106585.54  ],
       [1130785.92  ],
       [1188438.89  ],
       [1381330.35  ],
       [1342490.34  ],
       [1465263.51  ],
       [1339123.2278],
       [1339123.2278]])

In [97]:
def forecast_plot(observed, predicted):

    max_year = alameda_target.index.max()
    observed.loc[max_year+1] = predicted
    
    observations = observed[observed.index <= max_year]
    print(observations)
    predictions = observed[observed.index > max_year]
    
    fig = go.FigureWidget(data=[
    go.Scatter(x=observations.index, y=observations["Waste Produced (Tons)"], 
               mode='lines', line={'dash': 'solid'}, name="Observed"),
#     go.Scatter(x=predictions.index, y=predictions, 
#                mode='lines', line={'dash': 'dash'}, name="Predicted")
    ])
    return fig

In [98]:
forecast_plot(alameda_target, alameda_predicted)

        Waste Produced (Tons)
Year                         
2001.0           1.629208e+06
2002.0           1.596803e+06
2003.0           1.585191e+06
2004.0           1.692479e+06
2005.0           1.659523e+06
2006.0           1.656037e+06
2007.0           1.551103e+06
2008.0           1.337916e+06
2009.0           1.246311e+06
2010.0           1.152324e+06
2011.0           1.090343e+06
2012.0           1.147859e+06
2013.0           1.143318e+06
2014.0           1.106586e+06
2015.0           1.130786e+06
2016.0           1.188439e+06
2017.0           1.381330e+06
2018.0           1.342490e+06
2019.0           1.465264e+06
2020.0           1.339123e+06
2021.0           1.339123e+06
2022.0           1.339123e+06


FigureWidget({
    'data': [{'line': {'dash': 'solid'},
              'mode': 'lines',
              'name': '…

In [77]:
alameda_target.index.max()

2020.0

In [74]:
alameda_target.loc[2020] = alameda_predicted

In [75]:
alameda_target

,Waste Produced (Tons)
Year,
2001.0,1.629208e+06
2002.0,1.596803e+06
2003.0,1.585191e+06
2004.0,1.692479e+06
2005.0,1.659523e+06
2006.0,1.656037e+06
2007.0,1.551103e+06
2008.0,1.337916e+06
2009.0,1.246311e+06


## Multi year forecast

In [ ]:

average_waste_produced = 1.388987e+06
years_to_predict = np.arange(2020, 2025)
12 ** (years_to_predict % 2019 ) + average_waste_produced

complete_feature_df = pd.read_csv(DATA_PATH + "complete_feature_df.csv")

# average_waste_produced_county = complete_feature_df.groupby("County").agg(np.mean)[["Waste Produced (Tons)"]]
# average_waste_produced_county.head()

years_to_predict = np.arange(2020, 2025)
for county in average_waste_produced_county.index:
    average_waste_produced = average_waste_produced_county.loc[county, "Waste Produced (Tons)"]
    county_name = np.repeat(county, len(years_to_predict))
    dummy_predictions = 12 ** (years_to_predict % 2019) + average_waste_produced
#     average = np.repeat(average_waste_produced, len(years_to_predict))
    df = pd.DataFrame({"Year": years_to_predict, 
                       "County": county_name,
                       "Waste Produced (Tons)": dummy_predictions
                      })
    complete_feature_df = complete_feature_df.append(df, sort=False)

assert complete_feature_df.Year.max() == 2024

timeforecast_predictions = complete_feature_df[["Year", "County", "Waste Produced (Tons)"]]
timeforecast_predictions.sort_values(["Year", "County"], inplace=True)
timeforecast_predictions.head(3)

timeforecast_predictions.to_csv(DATA_PATH + "timeforecast_predictions.csv", index=False)